# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
# Imports
import pandas as pd
import requests
import json

In [12]:
def api_request(url,headers=None,params=None):
    """
    Send an HTTP GET request to the specified URL with optional headers and parameters.

    Args:
        url (str): The URL to send the GET request to.
        headers (dict, optional): A dictionary of HTTP headers to include in the request.
            Default is None.
        params (dict, optional): A dictionary of query parameters to include in the request.
            Default is None.
    """
    
    request = requests.get(url, headers=headers)
    return request.json()

def citybikes_city_search(api_cities, target_city):
    """
    Search for a specific city in a list of city network information and return its 'href'.

    This function iterates through a list of city network information, typically obtained from an API,
    and looks for a city whose 'location' matches the specified 'target_city'. If a matching city is found,
    the function returns the 'href' attribute of that city.

    Parameters:
    api_cities (dict): A dictionary containing a list of city network information, obtained from city_bikes API.
    target_city (str): The name of the city to search for, e.g., 'Los Angeles, CA'.

    Returns:
    str or None: The 'href' attribute of the matching city if found, or None if no match is found.
    """
    
    for city in api_cities['networks']:
        if city['location']['city'] == target_city:
           return city['href']

# Define the API endpoints for city information and station information.
url_city = 'http://api.citybik.es/v2/networks'
url_stations = f'http://api.citybik.es{citybikes_city_search(api_request(url_city), "Los Angeles, CA")}'



Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [13]:
station_info = api_request(url_stations)['network']['stations']

Put your parsed results into a DataFrame.

In [18]:
# create a dataframe with info for each station
bikes_df = pd.json_normalize(station_info)
bikes_df = bikes_df.rename(columns={'name':'station_address', 'latitude':'bike_station_lat','longitude':'bike_station_long'})
bikes_df = bikes_df[['station_address','bike_station_lat','bike_station_long','free_bikes']]

# Checking for null values
bikes_df.isnull().value_counts()

# Checking for duplicates
bikes_df.duplicated().value_counts()

# Confirming datatypes
bikes_df.dtypes

bikes_df.to_csv('bikes_df.csv', index=False)
